# FLASK APP

In [ ]:
from werkzeug.wrappers import Request, Response
import numpy as np
import cv2
import time
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from datetime import datetime
from deepface import DeepFace
from keras.preprocessing import image
import MySQLdb
conn = MySQLdb.connect(host="localhost", user="root", passwd="root", db="splatz")


In [ ]:
cd "C:\Users\Lenovo\Documents\PROJECT"

In [ ]:
train_dir = os.path.join(os.getcwd(),"train")
validation_dir = os.path.join(os.getcwd(),"test")


train_angry_dir = os.path.join(train_dir, 'angry')  # directory with our training angry pictures
train_disgust_dir = os.path.join(train_dir, 'disgust')  # directory with our training disgust pictures
train_fear_dir = os.path.join(train_dir, 'fear')  # directory with our training fear pictures
train_happy_dir = os.path.join(train_dir, 'happy')  # directory with our training happy pictures
train_neutral_dir = os.path.join(train_dir, 'neutral')  # directory with our training neutral pictures
train_sad_dir = os.path.join(train_dir, 'sad')  # directory with our training sad pictures
train_surprise_dir = os.path.join(train_dir, 'surprise')  # directory with our training surprise pictures


validation_angry_dir = os.path.join(validation_dir, 'angry')  # directory with our training angry pictures
validation_disgust_dir = os.path.join(validation_dir, 'disgust')  # directory with our training disgust pictures
validation_fear_dir = os.path.join(validation_dir, 'fear')  # directory with our training fear pictures
validation_happy_dir = os.path.join(validation_dir, 'happy')  # directory with our training happy pictures
validation_neutral_dir = os.path.join(validation_dir, 'neutral')  # directory with our training neutral pictures
validation_sad_dir = os.path.join(validation_dir, 'sad')  # directory with our training sad pictures
validation_surprise_dir = os.path.join(validation_dir, 'surprise') 

emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy',optimizer="Adam",metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        color_mode='grayscale',
        batch_size=64,
        class_mode='categorical')
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(48,48),
        color_mode='grayscale',
        batch_size=64,
        class_mode='categorical')

epochs=50
history = emotion_model.fit(
    train_generator,
    steps_per_epoch=28709// 64,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=7178//64
)
emotion_model.save('model_2.h5')

In [ ]:
from flask import Flask, render_template, redirect, request, session, jsonify,json,url_for
from flask_session import Session
import jinja2
app = Flask(__name__,static_folder='static')
my_loader = jinja2.ChoiceLoader([app.jinja_loader,jinja2.FileSystemLoader('static'),])
app.jinja_loader=my_loader
@app.route("/")
def hello():
    return render_template("index.html")

@app.route("/About")
def About():
    return render_template("about.html")

@app.route("/Contact")
def Contact():
    return render_template("contactus.html")

@app.route("/Adminlogin",methods=['POST'])
def Adminlogin():
    to_predict_list = request.form.to_dict()
    to_predict_list=list(to_predict_list.values())
    if to_predict_list[0] == 'Bhanu' and to_predict_list[1] == "bsp":
        return render_template("Adminlogin.html")
    else:
        return render_template("index.html")
    
@app.route("/Adminlogin/Register")
def register():
    return render_template("register.html")

@app.route("/Adminlogin/Register/Details",methods=['POST'])
def Details():
    to_predict_list = request.form.to_dict()
    to_predict_list=list(to_predict_list.values())
    userid = to_predict_list[0]
    name = to_predict_list[1]
    path = to_predict_list[2]
    password = to_predict_list[3]
    cursor = conn.cursor()
    mySql_insert_query = """INSERT INTO student VALUES (%s, %s, %s, %s) """
    record = (userid, name, path, password)
    cursor.execute(mySql_insert_query, record)
    conn.commit()
    print("record inserted")
    return render_template("Adminlogin.html")

@app.route("/Studentlogin",methods=['POST'])
def Studentlogin():
    to_predict_list = request.form.to_dict()
    to_predict_list=list(to_predict_list.values())
    name= to_predict_list[0]
    password = to_predict_list[1]
    cursor = conn.cursor()
    query="""SELECT * FROM student where name= %s and password=%s"""
    r = (name,password)
    cursor.execute(query,r)
    rec = cursor.fetchone()
    if rec == None:
        return render_template("index.html")
    else:
        print(rec,len(rec))
        session['record']=rec
        return render_template("Studentlogin.html",record=rec)

@app.route("/Studentlogin/Learn")
def Learn():
    return render_template("learn.html")

@app.route("/Studentlogin/Exam")
def Exam():
    return render_template("exam.html")

@app.route("/Studentlogin/Exam/Quiz1")
def Quiz1():
    return render_template("Quiz1.html")


@app.route("/Studentlogin/Exam/Quiz1/Task")
def seqt():
    cv2.ocl.setUseOpenCL(False)
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
    cap = cv2.VideoCapture(0)
    def make_1080p():
        cap.set(3, 1920)
        cap.set(4, 1080)
    def change_res(width, height):
        cap.set(3, width)
        cap.set(4, height)
    make_1080p()
    change_res(1280, 720)
    p=[]
    p1=[]
    now1 = datetime.now().strftime("%H:%M:%S")
    session['now1'] = now1
    n1 = now1
    FMT = '%H:%M:%S'
    now2 = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in num_faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray_frame = gray_frame[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
            emotion_prediction = emotion_model.predict(cropped_img)
            maxindex = int(np.argmax(emotion_prediction))
            n2 = datetime.now().strftime("%H:%M:%S")
            tdelta = datetime.strptime(n2, FMT) - datetime.strptime(n1, FMT)
            if tdelta.seconds > 5:
                    n1 = n2
                    p.append(maxindex)
                    pat = session.get("record")
                    pat = pat[2]
                    print(pat,now1,p,p1)
                    pat = ((str)(pat))
                    img1 = cv2.imread(r"C:\Users\Lenovo\Documents\PROJECT\static\UserImages"+"/"+pat)
                    photo= "img2.jpg"
                    face = frame[y:y+h,x:x+w]
                    cv2.imwrite(photo,face)
                    img2 = cv2.imread(r"C:\Users\Lenovo\Documents\PROJECT\img2.jpg")
                    plt.imshow(img2[:,:,::-1])
                    plt.show()
                    result = DeepFace.verify(img1,img2,enforce_detection=False)
                    p1.append(result["verified"])
            cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            now2 = datetime.now().strftime("%H:%M:%S")
            session['now2'] =now2
            session['p']=p
            session['p1'] = p1
            cap.release()
            cv2.destroyAllWindows()
            break 
    return redirect(url_for('temp1'))    

@app.route("/temp1")
def temp1():
    p1 = session.get('p1')
    p = session.get('p')
    now1  = session.get('now1')
    now2 = session.get('now2')
    print("hello",p,p1)
    if(p1.count(True)<=(len(p1)//2)):
        name = session.get("record")
        name = name[1]
        print(name)
        emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
        emo = p.count(0)
        val =0
        angrycount = p.count(0)
        angrypercent = (angrycount/len(p))*100
        
        disgustcount = p.count(1)
        disgustpercent = (disgustcount/len(p))*100
        if emo < disgustcount:
            val =1
            
        fearcount = p.count(2)
        fearpercent = (fearcount/len(p))*100
        if emo < fearcount:
            val =2
            
        happycount = p.count(3)
        happypercent = (happycount/len(p))*100
        print("Happy:",happypercent)
        if emo < happycount:
            val =3
            
        neutralcount = p.count(4)
        neutralpercent = (neutralcount/len(p))*100
        print("Neutral:",neutralpercent)
        if emo < neutralcount:
            val =4
            
        sadcount = p.count(5)
        sadpercent = (sadcount/len(p))*100
        print("Sad:",sadpercent)
        if emo < sadcount:
            val =5
            
        surprisedcount = p.count(6)
        surprisedpercent = (surprisedcount/len(p))*100
        print("Surprise:",surprisedpercent)
        if emo < surprisedcount:
            val =6
            
        print("Maximum emotion:",emotion_dict[val]) 
        emotion = ((str)(emotion_dict[val]))
        FMT = '%H:%M:%S'
        tdelta = datetime.strptime(now2, FMT) - datetime.strptime(now1, FMT)
        sec = ((str)(tdelta.seconds))
        cursor = conn.cursor()
        mySql_insert_query = """INSERT INTO exam VALUES (%s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s) """
        record = (name,'quiz1',angrypercent,disgustpercent,fearpercent,happypercent,neutralpercent,sadpercent,surprisedpercent,emotion,sec)
        cursor.execute(mySql_insert_query, record)
        conn.commit()
        print("Inserted successfully")
      
    return render_template("exam.html")

@app.route("/Performance1")
def Performanc():
    cursor = conn.cursor()
    cursor.execute("""SELECT * FROM exam""")
    rows = cursor.fetchall()
    return render_template("examperformance.html",value=rows)


@app.route("/Studentlogin/Learn/Topic1")
def Topic1():
    return render_template("Topic1.html")

@app.route("/Performance")
def Performance1():
    cursor = conn.cursor()
    cursor.execute("""SELECT * FROM history""")
    rows = cursor.fetchall()
    return render_template("learnperformance.html",value=rows)

@app.route("/Studentlogin/Learn/Topic1/Task")
def task():
    cv2.ocl.setUseOpenCL(False)
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
    cap = cv2.VideoCapture(0)
    def make_1080p():
        cap.set(3, 1920)
        cap.set(4, 1080)
    def change_res(width, height):
        cap.set(3, width)
        cap.set(4, height)
    make_1080p()
    change_res(1280, 720)
    p=[]
    p1=[]
    now1 = datetime.now().strftime("%H:%M:%S")
    session['now1'] = now1
    n1 = now1
    FMT = '%H:%M:%S'
    now2 = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in num_faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray_frame = gray_frame[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
            emotion_prediction = emotion_model.predict(cropped_img)
            maxindex = int(np.argmax(emotion_prediction))
            n2 = datetime.now().strftime("%H:%M:%S")
            tdelta = datetime.strptime(n2, FMT) - datetime.strptime(n1, FMT)
            if tdelta.seconds > 5:
                    n1 = n2
                    p.append(maxindex)
                    pat = session.get("record")
                    pat = pat[2]
                    print(pat,now1,p,p1)
                    pat = ((str)(pat))
                    img1 = cv2.imread(r"C:\Users\Lenovo\Documents\PROJECT\static\UserImages"+"/"+pat)
                    photo= "img2.jpg"
                    face = frame[y:y+h,x:x+w]
                    cv2.imwrite(photo,face)
                    img2 = cv2.imread(r"C:\Users\Lenovo\Documents\PROJECT\img2.jpg")
                    plt.imshow(img2[:,:,::-1])
                    plt.show()
                    result = DeepFace.verify(img1,img2,enforce_detection=False)
                    p1.append(result["verified"])
            cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            now2 = datetime.now().strftime("%H:%M:%S")
            session['now2'] =now2
            session['p']=p
            session['p1'] = p1
            cap.release()
            cv2.destroyAllWindows()
            break 
    return redirect(url_for('temp'))    

       
@app.route("/temp")
def temp():
    print("hello")
    p1 = session.get('p1')
    p = session.get('p')
    now1  = session.get('now1')
    now2 = session.get('now2')
    print("hello",p,p1)
    if(p1.count(True)<=(len(p1)//2)):
        name = session.get("record")
        name = name[1]
        print(name)
        emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
        emo = p.count(0)
        val =0
        angrycount = p.count(0)
        angrypercent = (angrycount/len(p))*100
        
        disgustcount = p.count(1)
        disgustpercent = (disgustcount/len(p))*100
        if emo < disgustcount:
            emo = disgustcount
            val =1
            
        fearcount = p.count(2)
        fearpercent = (fearcount/len(p))*100
        if emo < fearcount:
            emo = fearcount
            val =2
            
        happycount = p.count(3)
        happypercent = (happycount/len(p))*100
        print("Happy:",happypercent)
        if emo < happycount:
            emo = happycount
            val =3
            
        neutralcount = p.count(4)
        neutralpercent = (neutralcount/len(p))*100
        print("Neutral:",neutralpercent)
        if emo < neutralcount:
            emo = neutralcount
            val =4
            
        sadcount = p.count(5)
        sadpercent = (sadcount/len(p))*100
        print("Sad:",sadpercent)
        if emo < sadcount:
            emo = sadcount
            val =5
            
        surprisedcount = p.count(6)
        surprisedpercent = (surprisedcount/len(p))*100
        print("Surprise:",surprisedpercent)
        if emo < surprisedcount:
            emo = surprisedcount
            val =6
            
        print("Maximum emotion:",emotion_dict[val]) 
        emotion = ((str)(emotion_dict[val]))
        FMT = '%H:%M:%S'
        tdelta = datetime.strptime(now2, FMT) - datetime.strptime(now1, FMT)
        sec = ((str)(tdelta.seconds))
        cursor = conn.cursor()
        mySql_insert_query = """INSERT INTO history VALUES (%s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        if (tdelta.seconds)>(125//2):
            record = (name,'topic1',angrypercent,disgustpercent,fearpercent,happypercent,neutralpercent,sadpercent,surprisedpercent,emotion,sec,'Present')
            cursor.execute(mySql_insert_query, record)
            conn.commit()
            print("Inserted successfully")
        else:
            record = (name,"toipic1",angrypercent,  disgustpercent,fearpercent,happypercent,neutralpercent,sadpercent,surprisedpercent,emotion,sec,'Absent')
            cursor.execute(mySql_insert_query, record)
            conn.commit()
            print("Inserted successfully")
    return render_template("learn.html")
    

if __name__ == '__main__':
    app.secret_key = 'super secret key'
    app.config['SESSION_TYPE'] = 'filesystem'
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)